<a href="https://colab.research.google.com/github/nhamhung/Coder-School-Machine-Learning/blob/master/Week2_Tiki_Web_Scraping_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import sqlite3

TIKI_URL = 'https://tiki.vn'

In [0]:
conn = sqlite3.connect('tiki.db')
cur = conn.cursor()

In [0]:
# Create table categories in the database
def create_categories_table():
    query = """
        CREATE TABLE IF NOT EXISTS categories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            url TEXT, 
            parent_id INTEGER, 
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)
create_categories_table()

In [0]:
# Insert a row of data
# query = """
#     INSERT INTO categories (name, url, parent_id)
#     VALUES (?, ?, ?);
# """
# # val = ('test','test_url', 1)
# try:
#     cur.execute(query, val)
#     cat_id = cur.lastrowid
# except Exception as err:
#     print('ERROR BY INSERT:', err)

In [0]:
# cur.execute('SELECT * FROM categories;').fetchall()

In [0]:
# cur.execute('DROP TABLE categories;')

In [0]:
# Create a class Category
# attributes: name, url, parent_id
# instance method: save_into_db()
class Category:
    def __init__(self, name, url, parent_id=None, cat_id=None):
        self.cat_id = cat_id
        self.name = name
        self.url = url
        self.parent_id = parent_id

    def __repr__(self):
        return f"ID: {self.cat_id}, Name: {self.name}, URL: {self.url}, Parent: {self.parent_id}"

    def save_into_db(self):
        query = """
            INSERT INTO categories (name, url, parent_id)
            VALUES (?, ?, ?);
        """
        val = (self.name, self.url, self.parent_id)
        try:
            cur.execute(query, val)
            self.cat_id = cur.lastrowid
        except Exception as err:
            print('ERROR BY INSERT:', err)

# cat1 = Category('Test', 'Test URL')
# cat1.save_into_db()
# print(cat1.cat_id)
# cur.execute('SELECT * FROM categories;').fetchall()

In [0]:
# print(cat1)

In [0]:
# Get the HTML content get_url()
def get_url(url):
    try:
        response = requests.get(url).text
        soup = BeautifulSoup(response, 'html.parser')
        return soup
    except Exception as err:
        print('ERROR BY REQUEST:', err)
# get_url(TIKI_URL)

In [0]:
# get_main_categories()
def get_main_categories(save_db=False):
    soup = get_url(TIKI_URL)

    result = []
    for a in soup.find_all('a', {'class': 'MenuItem__MenuLink-sc-181aa19-1 fKvTQu'}):
        name = a.find('span', {'class': 'text'}).text
        url = a['href']
        main_cat = Category(name, url)

        if save_db:
            main_cat.save_into_db()
        result.append(main_cat)
    return result

In [0]:
main_categories = get_main_categories(save_db=True)
main_categories

[ID: 1, Name: Điện Thoại - Máy Tính Bảng, URL: https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner, Parent: None,
 ID: 2, Name: Điện Tử - Điện Lạnh, URL: https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner, Parent: None,
 ID: 3, Name: Phụ Kiện - Thiết Bị Số, URL: https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner, Parent: None,
 ID: 4, Name: Laptop - Thiết bị IT, URL: https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner, Parent: None,
 ID: 5, Name: Máy Ảnh - Quay Phim, URL: https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner, Parent: None,
 ID: 6, Name: Điện Gia Dụng, URL: https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner, Parent: None,
 ID: 7, Name: Nhà Cửa Đời Sống, URL: https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner, Parent: None,
 ID: 8, Name: Hàng Tiêu Dùng - Thực Phẩm

In [0]:
# get_sub_categories() given a parent category
import re

def get_sub_categories(parent_category, save_db=False):
    url = parent_category.url
    result = []

    try:
        soup = get_url(url)
        div_containers = soup.find_all('div', {'class':'list-group-item is-child'})
        for div in div_containers:
            name = div.a.text
            name = re.sub('\s{2,}', ' ', name)
            url = TIKI_URL + div.a['href']
            cat = Category(name, url, parent_category.cat_id)
            if save_db:
                cat.save_into_db()
            result.append(cat)
    except Exception as err:
        print('ERROR BY GET SUB CATEGORIES:', err)
    return result

In [0]:
get_sub_categories(main_categories[0], save_db=True)


[ID: 3240, Name:  Máy tính bảng (51)
 , URL: https://tiki.vn/may-tinh-bang/c1794?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1,
 ID: 3241, Name:  Máy đọc sách (29)
 , URL: https://tiki.vn/may-doc-sach/c28856?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1,
 ID: 3242, Name:  Điện thoại Smartphone (191)
 , URL: https://tiki.vn/dien-thoai-smartphone/c1795?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1,
 ID: 3243, Name:  Điện thoại bàn (108)
 , URL: https://tiki.vn/dien-thoai-ban/c8061?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1,
 ID: 3244, Name:  Điện thoại phổ thông (90)
 , URL: https://tiki.vn/dien-thoai-pho-thong/c1796?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1]

In [0]:
# get_all_categories() given a list of main categories
data = []

def get_all_categories(categories):
    if len(categories) == 0:
        # products = soup.find_all('div', {'class':'product_item'})

        # for product in products:
        #     d = {"Product Title":"", "Product Price":""}

        #     try:
        #         d["Product Title"] = product['data-title']
        #         d["Product Price"] = product['data-price']
        #         data.append(d)
        #     except:
        #         pass
        return
    for cat in categories:
        sub_categories = get_sub_categories(cat, save_db=True)
        print(sub_categories)
        get_all_categories(sub_categories)



In [0]:
get_all_categories(main_categories)


Streaming output truncated to the last 5000 lines.
[ID: 758, Name:  Bàn học sinh (377)
, URL: https://tiki.vn/ban-hoc-sinh/c4380?src=c.1883.hamburger_menu_fly_out_banner, Parent: 753, ID: 759, Name:  Bàn làm việc (719)
, URL: https://tiki.vn/ban-lam-viec/c4381?src=c.1883.hamburger_menu_fly_out_banner, Parent: 753, ID: 760, Name:  Bàn vi tính (37)
, URL: https://tiki.vn/ban-vi-tinh/c23656?src=c.1883.hamburger_menu_fly_out_banner, Parent: 753, ID: 761, Name:  Ghế làm việc (767)
, URL: https://tiki.vn/ghe-lam-viec/c12592?src=c.1883.hamburger_menu_fly_out_banner, Parent: 753, ID: 762, Name:  Nội thất phòng học, làm việc khác (80)
, URL: https://tiki.vn/noi-that-phong-hoc-lam-viec-khac/c23660?src=c.1883.hamburger_menu_fly_out_banner, Parent: 753, ID: 763, Name:  Tủ, kệ sách (652)
, URL: https://tiki.vn/tu-ke-sach/c23658?src=c.1883.hamburger_menu_fly_out_banner, Parent: 753]
[]
[]
[]
[]
[]
[]
[ID: 764, Name:  Bàn sofa/salon (554)
, URL: https://tiki.vn/ban-sofasalon/c23576?src=c.1883.hamburg

In [0]:
# To get the product, we need to get the lowest level categories and crawl from their urls (SELECT query)
import pandas as pd
list_of_categories = cur.execute('SELECT * FROM categories;').fetchall()
list_of_categories
df = pd.DataFrame(list_of_categories, columns = ['cat_id', 
                                       'Name', 
                                       'URL',
                                       'parent_id', 
                                       'time'])
df

,cat_id,Name,URL,parent_id,time
0,1,Điện Thoại - Máy Tính Bảng,https://tiki.vn/dien-thoai-may-tinh-bang/c1789...,NaN,2020-05-31 08:04:20
1,2,Điện Tử - Điện Lạnh,https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?...,NaN,2020-05-31 08:04:20
2,3,Phụ Kiện - Thiết Bị Số,https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815...,NaN,2020-05-31 08:04:20
3,4,Laptop - Thiết bị IT,https://tiki.vn/laptop-may-vi-tinh/c1846?src=c...,NaN,2020-05-31 08:04:20
4,5,Máy Ảnh - Quay Phim,https://tiki.vn/may-anh/c1801?src=c.1801.hambu...,NaN,2020-05-31 08:04:20
...,...,...,...,...,...
3234,3235,GYM (23)\n,https://tiki.vn/gym/c13366?src=c.11312.hamburg...,3231.0,2020-05-31 08:40:21
3235,3236,Yoga (4)\n,https://tiki.vn/yoga/c13362?src=c.11312.hambur...,3231.0,2020-05-31 08:40:21
3236,3237,Ca nhạc - Phim - Kịch (4)\n,https://tiki.vn/ca-nhac-phim-kich/c11328?src=c...,3196.0,2020-05-31 08:40:24
3237,3238,Studio - Chụp ảnh (47)\n,https://tiki.vn/studio-chup-anh/c11330?src=c.1...,3196.0,2020-05-31 08:40:24


In [0]:
list_of_URLs = []

for url in df['URL']:
    soup = get_url(url)

    # print(soup)
    try:
        if soup.find_all('div', {'class':'list-group-item is-child'}):
            print('Parent URL: ', url)
        else:
            list_of_URLs.append(url)
            print("Child URL: ", url)
    except:
        pass

list_of_URLs



Parent URL:  https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner
Parent URL:  https://tiki.vn/thoi-trang/c914?src=c.914.hamburger_m

['https://tiki.vn/may-tinh-bang/c1794?src=c.1789.hamburger_menu_fly_out_banner',
 'https://tiki.vn/may-doc-sach/c28856?src=c.1789.hamburger_menu_fly_out_banner',
 'https://tiki.vn/dien-thoai-smartphone/c1795?src=c.1789.hamburger_menu_fly_out_banner',
 'https://tiki.vn/dien-thoai-ban/c8061?src=c.1789.hamburger_menu_fly_out_banner',
 'https://tiki.vn/dien-thoai-pho-thong/c1796?src=c.1789.hamburger_menu_fly_out_banner',
 'https://tiki.vn/may-giat/c3862?src=c.4221.hamburger_menu_fly_out_banner',
 'https://tiki.vn/may-lanh-may-dieu-hoa/c3865?src=c.4221.hamburger_menu_fly_out_banner',
 'https://tiki.vn/may-nuoc-nong/c3866?src=c.4221.hamburger_menu_fly_out_banner',
 'https://tiki.vn/may-rua-chen/c3864?src=c.4221.hamburger_menu_fly_out_banner',
 'https://tiki.vn/may-say-quan-ao/c3863?src=c.4221.hamburger_menu_fly_out_banner',
 'https://tiki.vn/tu-lanh/c2328?src=c.4221.hamburger_menu_fly_out_banner',
 'https://tiki.vn/tu-dong-tu-mat/c3868?src=c.4221.hamburger_menu_fly_out_banner',
 'https://tik

In [0]:
len(list_of_URLs)


2670

[]

In [0]:
# Create table products in the database

def create_products_table():
    query = """
        CREATE TABLE IF NOT EXISTS products (
            product_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            url TEXT, 
            parent_url TEXT, 
            tiki_product_id INTEGER,
            original_price INTEGER,
            discount_price INTEGER,
            review VARCHAR(255),
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)


In [0]:
class Product:
    def __init__(self, name, url, parent_url, tiki_product_id, original_price, discount_price, review, product_id=None):
        self.product_id = product_id
        self.name = name
        self.url = url
        self.parent_url = parent_url
        self.tiki_product_id = tiki_product_id
        self.original_price = original_price
        self.discount_price = discount_price
        self.review = review

    def __repr__(self):
        return f"ID: {self.product_id}, Name: {self.name}, URL: {self.url}, Parent: {self.category_id}"

    def save_into_db(self):
        query = """
            INSERT INTO products (name, url, parent_url, tiki_product_id, original_price, discount_price, review)
            VALUES (?, ?, ?, ?, ?, ?, ?);
        """
        val = (self.name, self.url, self.parent_url, self.tiki_product_id, self.original_price, self.discount_price, self.review)
        try:
            cur.execute(query, val)
            self.product_id = cur.lastrowid
        except Exception as err:
            print('ERROR BY INSERT:', err)


In [0]:
import re

def get_all_products(url):

    i = 1

    new_url = url + '&page=' + str(i)
    soup = get_url(new_url)

    data = []
    while soup.find_all('div', {'class':'product-item'}) != []:

        new_url = url + '&page=' + str(i)

        soup = get_url(new_url)

        products = soup.find_all('div', {'class':'product-item'})

        for product in products:
            try:
                dictionary = {'name':'', 'url':'', 'tiki_product_id':'', 'original_price':'', 'discount_price':'', 'review':''}

                p_name = product['data-title']
                dictionary['name'] = p_name

                p_url = product.a['href']
                dictionary['url'] = url

                p_tiki_id = product['data-seller-product-id']
                dictionary['tiki_product_id'] = p_tiki_id

                if product.find('span', {'class':'price-regular'}):
                    p_original_price = int(product.find('span', {'class':'price-regular'}).text.replace(' ', '').replace('.', '').replace('đ', ''))
                    dictionary['original_price'] = p_original_price
                else:
                    p_original_price = product['data-price']
                    dictionary['original_price'] = p_original_price

                p_discount_price = product['data-price']
                dictionary['discount_price'] = p_discount_price

                p_review = 0                    
                if "(" in product.find('p', {'class':'review'}).text:
                    p_review = product.find('p', {'class':'review'}).text
                    p_review = int(re.findall('\d+', p_review)[0])
                    dictionary['review'] = p_review
                else:
                    dictionary['review'] = p_review
                
                new_product = Product(p_name, p_url, url, p_tiki_id, p_original_price, p_discount_price, p_review)
                new_product.save_into_db()

                data.append(dictionary)
            
            except Exception as err:
                print('ERROR BY GET ALL PRODUCTS:', err)
        i += 1
    return data
        
# get_all_products('https://tiki.vn/may-tinh-bang/c1794?src=c.1789.hamburger_menu_fly_out_banner')



In [0]:
# get_all_products('https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner')

In [0]:
cur.execute('DROP TABLE products;').fetchall()
create_products_table()

In [0]:
def get_total_products():
    i = 0

    for url in list_of_URLs:
        try:
            print(f'{i} :', url)
            i += 1
            get_all_products(url)
        except:
            pass

get_total_products()


0 : https://tiki.vn/may-tinh-bang/c1794?src=c.1789.hamburger_menu_fly_out_banner
1 : https://tiki.vn/may-doc-sach/c28856?src=c.1789.hamburger_menu_fly_out_banner
2 : https://tiki.vn/dien-thoai-smartphone/c1795?src=c.1789.hamburger_menu_fly_out_banner
3 : https://tiki.vn/dien-thoai-ban/c8061?src=c.1789.hamburger_menu_fly_out_banner
4 : https://tiki.vn/dien-thoai-pho-thong/c1796?src=c.1789.hamburger_menu_fly_out_banner
5 : https://tiki.vn/may-giat/c3862?src=c.4221.hamburger_menu_fly_out_banner
6 : https://tiki.vn/may-lanh-may-dieu-hoa/c3865?src=c.4221.hamburger_menu_fly_out_banner
7 : https://tiki.vn/may-nuoc-nong/c3866?src=c.4221.hamburger_menu_fly_out_banner
8 : https://tiki.vn/may-rua-chen/c3864?src=c.4221.hamburger_menu_fly_out_banner
9 : https://tiki.vn/may-say-quan-ao/c3863?src=c.4221.hamburger_menu_fly_out_banner
10 : https://tiki.vn/tu-lanh/c2328?src=c.4221.hamburger_menu_fly_out_banner
11 : https://tiki.vn/tu-dong-tu-mat/c3868?src=c.4221.hamburger_menu_fly_out_banner
12 : https:

In [0]:
df = pd.read_sql_query('SELECT * FROM products;', conn)
df

,product_id,name,url,parent_url,tiki_product_id,original_price,discount_price,review,create_at
0,1,iPad 10.2 Inch WiFi 32GB New 2019 - Hàng Chín...,https://tiki.vn/ipad-10-2-inch-wifi-32gb-new-2...,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,54431583,9990000,8990000,0,2020-05-31 14:29:41
1,2,iPad 10.2 Inch WiFi 128GB New 2019 - Hàng Chí...,https://tiki.vn/ipad-10-2-inch-wifi-128gb-new-...,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,54438926,11990000,11290000,0,2020-05-31 14:29:41
2,3,iPad 10.2 Inch WiFi 32GB New 2019 - Hàng Nhập ...,https://tiki.vn/ipad-10-2-inch-wifi-32gb-new-2...,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,32648373,9990000,8690000,209,2020-05-31 14:29:41
3,4,iPad 10.2 Inch WiFi 128GB New 2019 - Hàng Nhập...,https://tiki.vn/ipad-10-2-inch-wifi-128gb-new-...,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,0,12990000,10990000,0,2020-05-31 14:29:41
4,5,iPad Pro 11 inch (2020) Wifi - Hàng Nhập Khẩu ...,https://tiki.vn/ipad-pro-11-inch-2020-wifi-han...,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,51428410,23990000,22990000,0,2020-05-31 14:29:41
...,...,...,...,...,...,...,...,...,...
647551,647552,Sim 4G Malaysia,https://tiki.vn/sim-4g-malaysia-p7166681.html?...,https://tiki.vn/vui-choi-giai-tri/c11329?src=c...,7166683,580000,450000,0,2020-05-31 17:56:15
647552,647553,Sim 4G Malaysia Gói 4GB Trong 15 Ngày,https://tiki.vn/sim-4g-malaysia-goi-4gb-trong-...,https://tiki.vn/vui-choi-giai-tri/c11329?src=c...,7166933,790000,600000,0,2020-05-31 17:56:15
647553,647554,Sim 4G Đài Loan Gói 4GB Trong 15 Ngày,https://tiki.vn/sim-4g-dai-loan-goi-4gb-trong-...,https://tiki.vn/vui-choi-giai-tri/c11329?src=c...,7166909,790000,599000,0,2020-05-31 17:56:15
647554,647555,Sim 4G Châu Âu,https://tiki.vn/sim-4g-chau-au-p7005711.html?s...,https://tiki.vn/vui-choi-giai-tri/c11329?src=c...,7005713,900000,690000,0,2020-05-31 17:56:15
